In [1]:
import numpy as np
import sys, os
#sys.path.insert(0,"../../qpms/")
from qpms import *
from qpms.cybspec import BaseSpec
from qpms.cytmatrices import CTMatrix
from qpms.qpms_c import Particle, Ewald3Calculator, pgsl_ignore_error, gamma_inc
from qpms.cymaterials import EpsMuGenerator, LorentzDrudeModel, lorentz_drude

# matrix "distance"
def array_reldiff(a, b):
    return np.amax(abs(a-b))/np.amax(abs(a)+abs(b))

# Analyticity test via Cauchy-Riemann eqs.
def anal_test(fun, z, dx=None, dy=None, rpos=0, ipos=0):
    if dx is None:
        dx = abs(z)*1e-8+1e-16
    if dy is None:
        dy = abs(z)*1e-8+1e-16
    zyp = z + dy*1j if ipos > 0 else z if ipos < 0 else z + dy*.5j
    zym = z if ipos > 0 else z - dy*1j if ipos < 0 else z - dy*.5j
    zxp = z + dx if ipos > 0 else z if ipos < 0 else z + dx*.5
    zxm = z if ipos > 0 else z - dx if ipos < 0 else z - dx*.5    
    dfdy = (fun(zyp) - fun(zym)) / dy
    dfdx = (fun(zxp) - fun(zxm)) / dx
    return array_reldiff(1j * dfdx, dfdy)
    #return (dfdx, dfdy)
    
    
    

In [3]:
c = Ewald3Calculator(3)
a1 = np.array([0,1.0E-6])
a2 = np.array([1.0E-6,0])
area = 1e-6
eta = 2.224/area
beta = np.array([0, 3900000])


In [4]:
wnplus = 12.3e6+0j
wnminus = 12.3e6-1j
print(c.sigma0(eta, wnplus))
print(c.sigma0(eta, wnminus))

(-0.9999999991836039-37.09631581406487j)
(-1.0000347332888828-37.096315813013305j)


In [5]:
splus = c.sigma_short(eta, wnplus, [a1,a2], beta, [0,0])
sminus = c.sigma_short(eta, wnminus, [a1,a2], beta, [0,0])
(splus-sminus)/(abs(splus)+abs(sminus))

/u/46/necadam1/unix/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()


array([ 5.32454017e-07-3.10588363e-13j, -4.88321139e-07+2.68003416e-13j,
                   nan           +nanj, -4.88321139e-07+2.68003416e-13j,
        4.42772646e-07-2.27166373e-13j,             nan           +nanj,
       -4.41774978e-07+2.26289431e-13j,             nan           +nanj,
        4.42772646e-07-2.27057836e-13j,  3.96070034e-07-1.90885387e-13j,
                   nan           +nanj,  3.96877102e-07-1.91576540e-13j,
                   nan           +nanj,  3.96877102e-07-1.91501735e-13j,
                   nan           +nanj,  3.96070034e-07-1.90885387e-13j])

In [6]:
with pgsl_ignore_error(15):
    lplus = c.sigma_long(eta, wnplus, [a1,a2], beta, [0,0], False)
    lminus = c.sigma_long(eta, wnminus, [a1,a2], beta, [0,0], False)
(lplus-lminus)/(abs(lplus)+abs(lminus))

/u/46/necadam1/unix/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:4: RuntimeWarning: invalid value encountered in true_divide


array([ 4.65732616e-07+0.99967727j, -3.55960180e-07-0.41212499j,
                   nan       +nanj, -3.55960497e-07-0.41212499j,
        7.21597934e-08+0.92915829j,             nan       +nanj,
       -2.86904829e-07+0.93544685j,             nan       +nanj,
        7.21598406e-08+0.92915829j,  4.93117296e-07+0.99195553j,
                   nan       +nanj,  5.10150007e-08+0.73932693j,
                   nan       +nanj,  5.10122654e-08+0.73932693j,
                   nan       +nanj,  4.93117355e-07+0.99195553j])

In [7]:
lplus, lminus

(array([ 0.94910028+3.72396414e+01j,  0.21761126-9.22333269e-02j,
         0.        +0.00000000e+00j,  0.21761126-9.22333269e-02j,
         0.71676485+2.14170631e+00j,  0.        +0.00000000e+00j,
        -0.01893024+1.66386040e-02j,  0.        +0.00000000e+00j,
         0.71676485+2.14170631e+00j,  0.29356288+2.05525877e+00j,
         0.        +0.00000000e+00j,  0.3430742 +3.85349725e-01j,
         0.        +0.00000000e+00j,  0.3430742 +3.85349725e-01j,
         0.        +0.00000000e+00j,  0.29356288+2.05525877e+00j]),
 array([ 0.94906548-37.45606086j,  0.21761143 +0.10469544j,
         0.         +0.j        ,  0.21761143 +0.10469544j,
         0.71676456 -1.50897322j,  0.         +0.j        ,
        -0.0189302  -0.12769375j,  0.         +0.j        ,
         0.71676456 -1.50897322j,  0.29356058 -2.57425425j,
         0.         +0.j        ,  0.34307415 -0.36814078j,
         0.         +0.j        ,  0.34307415 -0.36814078j,
         0.         +0.j        ,  0.29356058 -2.5

In [3]:
gamma_inc(1/2, -2), gamma_inc(1/2, -2-1e-14j)

(((1.7724538509055165-6.68768552562196j), 6.250055312558605e-13),
 ((1.77245385090546+6.68768552562196j), 6.250055312558604e-13))

In [4]:
gamma_inc(-1/2, -2), gamma_inc(-1/2, -2-1e-14j)

(((-3.5449077018110287+2.925667703000558j), 1.2817942489249855e-12),
 ((-3.5449077018110122-2.925667703000558j), 1.2817942489249853e-12))

In [5]:
gamma_inc(-3/2, -2), gamma_inc(-3/2, -2-1e-14j)

(((2.363271801207352-0.20882791062647624j), 8.624748866913507e-13),
 ((2.3632718012073406+0.20882791062647624j), 8.624748866913506e-13))